In [1]:
import os
import pandas as pd
from skimage import io
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms,utils
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class FaceDataset(Dataset):
    def __init__(self, csv_file, txt_file, root_dir, transform=None):
        self.index = pd.read_csv(csv_file)
        self.labels = np.loadtxt(txt_file)[:,3]
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        
        img_name = os.path.join(self.root_dir, self.index.iloc[idx, 0])
        image = io.imread(img_name)
        image = image.astype('float')
        labels = self.labels[idx]
        labels = labels.astype('long').reshape(-1)
        sample = {'image':image, 'label':labels}
        
        if self.transform:
            sample = self.transform(sample)
        
        return sample

class ToTensor(object):
    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        image = np.transpose(image, (2, 0, 1))
        label = (label + 1) / 2
        return {'image':torch.FloatTensor(image),
                 'label':torch.LongTensor(label)}

In [3]:
transformed_dataset = FaceDataset(csv_file='img/label.csv', txt_file='img/labelss.txt',
                                  root_dir='img/', transform=transforms.Compose([
    ToTensor()
]))
train_size = 4000
test_size = 1000
batch_size = 200
dataloader = DataLoader(transformed_dataset, batch_size=batch_size, shuffle=True)
train_dataset, test_dataset = torch.utils.data.random_split(transformed_dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=test_size, shuffle=False)

In [4]:
train_dataset[0]['label']

tensor([0])

In [5]:
# 建立卷积神经网络模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, 5, 1)
        self.conv2 = nn.Conv2d(10, 20, 4, 1)
        self.conv3 = nn.Conv2d(20, 50, 3, 1)
        self.fc1 = nn.Linear(25*20*50, 10000)
        self.fc2 = nn.Linear(10000, 1000)
        self.fc3 = nn.Linear(1000, 100)
        self.fc4 = nn.Linear(100, 2)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 25*20*50)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x,dim=1)

In [6]:
# 设置参数、模型、优化器
lr = 0.01
momentum = 0.5
model = Net()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
num_epochs = 2

In [7]:
# 训练函数
def train(model, train_loader, optimizer, epoch):
    model.train()
    for idx, sample in enumerate(train_loader): # idx:字典的批序号，sample:第idx批字典
        data = sample['image'] # data:tensor格式float32, 30 * 3 * 94 * 94
        target = sample['label'].view(-1) # target:tensor格式， 30 * 1
        data = torch.tensor(data, dtype=torch.float32)
        pred = model(data) # pred:tensor格式， 30 * 1
        loss = F.nll_loss(pred, target) # loss:tensor格式标量
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print("Train Epoch:{}, iteration:{}, Loss:{}".format(epoch, idx, loss.item()))

In [8]:
# 测试函数
def test(model, test_loader):
    model.eval()
    total_loss = 0.
    correct = 0.
    with torch.no_grad():
        for idx, sample in enumerate(test_loader): # idx:字典的批序号，sample:第idx批字典
            data = sample['image'] # data:tensor格式float32, 1 * 3 * 94 * 94
            target = sample['label'].view(-1) # target:tensor格式标量
            data = torch.tensor(data, dtype=torch.float32)
            output = model(data) # output:tensor格式标量
            total_loss = F.nll_loss(output, target, reduction = 'sum').item() # total_loss:tensor格式标量
            pred = output.argmax(dim = 1)
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    total_loss /= len(test_loader.dataset)
    acc = correct / len(test_loader.dataset) * 100
    print("Test Loss:{}, Test Accuracy:{}".format(total_loss, acc))

In [9]:
for epoch in range(num_epochs):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)

<ipython-input-7-2d6768df04eb>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data, dtype=torch.float32)


Train Epoch:0, iteration:0, Loss:0.6946545243263245
Train Epoch:0, iteration:1, Loss:0.9938757419586182
Train Epoch:0, iteration:2, Loss:0.7608091235160828
Train Epoch:0, iteration:3, Loss:0.714445948600769
Train Epoch:0, iteration:4, Loss:0.6898495554924011
Train Epoch:0, iteration:5, Loss:0.6899532079696655
Train Epoch:0, iteration:6, Loss:0.6953942775726318
Train Epoch:0, iteration:7, Loss:0.7011470794677734
Train Epoch:0, iteration:8, Loss:0.6904239058494568
Train Epoch:0, iteration:9, Loss:0.6894770264625549
Train Epoch:0, iteration:10, Loss:0.6895323991775513
Train Epoch:0, iteration:11, Loss:0.6897440552711487
Train Epoch:0, iteration:12, Loss:0.6901335120201111
Train Epoch:0, iteration:13, Loss:0.685885488986969
Train Epoch:0, iteration:14, Loss:0.6847776770591736
Train Epoch:0, iteration:15, Loss:0.6851357221603394
Train Epoch:0, iteration:16, Loss:0.6863030195236206
Train Epoch:0, iteration:17, Loss:0.6794758439064026
Train Epoch:0, iteration:18, Loss:0.6812039017677307
Train

<ipython-input-8-8b94175df9cd>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data, dtype=torch.float32)


Test Loss:0.6773395385742188, Test Accuracy:54.1
Train Epoch:1, iteration:0, Loss:0.6720249652862549


KeyboardInterrupt: 